In [6]:
import pandas as pd
import numpy as np
import datetime

allData = pd.read_csv("./data/train.csv")

In [9]:
allData.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


MISSING VALUES 
 Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64


### Data Cleaning

In [39]:
sizeFeatures = [
    "MSSubClass", "Utilities", "BldgType", "TotalBsmtSF", "LowQualFinSF",
    "GrLivArea", "FullBath", "HalfBath", "TotRmsAbvGrd",
    "Fireplaces", "GarageArea"
]

locationFeatures = [
    "MSZoning", "LotFrontage", "LotArea", "Street",
    "LandContour", "LotConfig", "LandSlope", "Neighborhood", "Condition1",
    "YearBuilt", "YearRemodAdd", "Heating", "Functional", "PavedDrive"
]

targetFeature = ["SalePrice"]

selectedData = allData[sizeFeatures + locationFeatures]
print(selectedData)

#types are ordered from worst to best
utilitiesTypes = ["ELO", "NoSeWa", "NoSeWr", "AllPub"]
bldgTypes = ["TwnhsI", "TwnhsE", "Duplx", "2FmCon", "1Fam"]
mszoningTypes = ["A","I","C", "FV", "RH", "RM", "RL", "RP"]
streetTypes = ["Grvl", "Pave"]
landcontourTypes = ["Low", "Lvl", "Bnk", "HLS"]
lotconfigTypes = ["Inside", "Corner", "FR2", "FR3", "CulDSac"]
landslopeTypes = ["Sev", "Mod", "Gtl"]
neighborhoodTypes = []






      MSSubClass Utilities BldgType  TotalBsmtSF  LowQualFinSF  GrLivArea  \
0             60    AllPub     1Fam          856             0       1710   
1             20    AllPub     1Fam         1262             0       1262   
2             60    AllPub     1Fam          920             0       1786   
3             70    AllPub     1Fam          756             0       1717   
4             60    AllPub     1Fam         1145             0       2198   
...          ...       ...      ...          ...           ...        ...   
1455          60    AllPub     1Fam          953             0       1647   
1456          20    AllPub     1Fam         1542             0       2073   
1457          70    AllPub     1Fam         1152             0       2340   
1458          20    AllPub     1Fam         1078             0       1078   
1459          20    AllPub     1Fam         1256             0       1256   

      FullBath  HalfBath  TotRmsAbvGrd  Fireplaces  ...  LandContour  \
0  

In [59]:
numericalFeatures = ["MSSubClass", "TotalBsmtSF", "LowQualFinSF",
    "GrLivArea", "FullBath", "HalfBath", "TotRmsAbvGrd",
    "Fireplaces", "GarageArea", "LotFrontage", "LotArea", "GrLivArea"]



count = 0
for column in numericalFeatures:
    selectedData.fillna({column: 0}, inplace=True)


#for column in sizeFeatures + locationFeatures:
    
    #if selectedData[column].dtype != 'int64' and selectedData[column].dtype != 'float64':
       #selectedData[column].map(int)


selectedData["Utilities"] =  selectedData["Utilities"].astype("category")
print(selectedData["Utilities"].dtype)
print("count is: " + str(count))








category
count is: 0


/tmp/ipykernel_16217/3351901058.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selectedData.fillna({column: 0}, inplace=True)
/tmp/ipykernel_16217/3351901058.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selectedData["Utilities"] =  selectedData["Utilities"].astype("category")


In [60]:
from sklearn.neighbors import KNeighborsRegressor
KNNregressor = KNeighborsRegressor(n_neighbors=2)
KNNregressor.fit(trainingData, testData)
KNeighborsRegressor(...)


ValueError: could not convert string to float: 'AllPub'